In [42]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Optional: Force external browser rendering
pio.renderers.default = "browser"

# Create wavelength axes
x = np.linspace(400, 1100, 701)
y = np.linspace(400, 1100, 701)
X, Y = np.meshgrid(x, y)

# Initialize Z with zeros
Z = np.zeros_like(X)

# Lock random seed
np.random.seed(42)

# Create multiple random diagonal peaks
num_diag_peaks = 3
for _ in range(num_diag_peaks):
    center = np.random.uniform(400, 1100)
    peak_height = np.random.uniform(400, 200)
    sigma = np.random.uniform(30, 80)
    Z += peak_height * np.exp(-(((X - center)**2 + (Y - center)**2) / (2 * sigma**2)))

# Create small off-diagonal random bumps
num_offdiag_peaks = 20
for _ in range(num_offdiag_peaks):
    center_x = np.random.uniform(400, 1100)
    center_y = np.random.uniform(400, 1100)
    if abs(center_x - center_y) < 30:
        continue
    peak_height = np.random.uniform(20, 100)
    sigma = np.random.uniform(20, 60)
    Z += peak_height * np.exp(-(((X - center_x)**2 + (Y - center_y)**2) / (2 * sigma**2)))

# Clip any negatives
Z = np.clip(Z, 0, None)

# Define plane region and height
plane_x_min = 750
plane_x_max = 875
plane_y_min = 750
plane_y_max = 875
plane_z = 250

# Mask for points within the plane region AND above plane_z
mask_red = (
    (X >= plane_x_min) & (X <= plane_x_max) &
    (Y >= plane_y_min) & (Y <= plane_y_max) &
    (Z >= plane_z)
)

# Diagnostics: how many points satisfy this?
print("Number of red points:", np.sum(mask_red))

# Also check the max Z in the box to confirm
in_box = (
    (X >= plane_x_min) & (X <= plane_x_max) &
    (Y >= plane_y_min) & (Y <= plane_y_max)
)
print("Max Z in box:", np.max(Z[in_box]))

# Original surface
surface_original = go.Surface(
    x=X,
    y=Y,
    z=Z,
    colorscale='Viridis',
    cmin=0,
    cmax=700,
    showscale=False,
    opacity=1
)

# Compute normalized Z in the masked region
Z_in_mask = Z[mask_red]
if len(Z_in_mask) > 0:
    min_z = np.min(Z_in_mask)
    max_z = np.max(Z_in_mask)
    norm_Z = (Z - min_z) / (max_z - min_z + 1e-9)
    norm_Z = np.clip(norm_Z, 0, 1)
else:
    norm_Z = np.zeros_like(Z)

# Prepare surface with gradient colors
Z_gradient = np.where(mask_red, Z, np.nan)
norm_Z_gradient = np.where(mask_red, norm_Z, np.nan)

surface_gradient = go.Surface(
    x=X,
    y=Y,
    z=Z_gradient,
    surfacecolor=norm_Z_gradient,  # Use normalized Z to drive color
    colorscale=[
        [0.0, 'yellow'],
        [1.0, 'red']
    ],
    showscale=False,
    opacity=1
)

# Plane at Z=300
Z_plane = np.full_like(Z, plane_z)
Z_plane = np.where(
    (X >= plane_x_min) & (X <= plane_x_max) &
    (Y >= plane_y_min) & (Y <= plane_y_max),
    Z_plane,
    np.nan
)

surface_plane = go.Surface(
    x=X,
    y=Y,
    z=Z_plane,
    colorscale=[[0, 'skyblue'], [1, 'skyblue']],
    showscale=False,
    opacity=0.5
)

# Create text marker at (820,820) with z~400
x_coord = 820
y_coord = 820
z_text = 380

text_marker = go.Scatter3d(
    x=[x_coord],
    y=[y_coord],
    z=[z_text + 15],  # offset above the surface
    mode='text',
    text=(f"Wavelength 515 nm"),
    textposition="top center",
    textfont=dict(color='green', size=20)
)

layout = go.Layout(
    title='Simulated 3D Mountain with Red Highlight Above Plane Z=300',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    autosize=True,
    margin=dict(l=0, r=0, b=0, t=0)
)

# Important: draw red surface last so it is on top
fig = go.Figure(
    data=[surface_original, surface_plane, surface_gradient, text_marker],
    layout=layout
)
fig.show()


Number of red points: 3912
Max Z in box: 379.78983435604204
